In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import california_housing

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold

import matplotlib.pyplot as plt

    Задача: построить алгоритма градиентного бустинга с квадратичной функцией потерь, в качестве базового алгоритма использовать алгоритм CART.
   

In [50]:
class MyDecisionTreeRegr:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1,
                 min_weight_fraction_leaf=0.0, max_features=None, random_state=None,
                 max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_weight_fraction_leaf = min_weight_fraction_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.max_leaf_nodes = max_leaf_nodes
        self.min_impurity_decrease = min_impurity_decrease
        self.min_impurity_split = min_impurity_split
        
        self.tree = dict()  # Номер узла: информация об узле.
        
        self.Leaf = True
        self.notLeaf = False
        
    def _calculate_impurity(self, y):
        return np.var(y)
        
    def _search_split(self, X, y):
        best_criterion = float('-inf')
        best_feature = None
        best_threshold = None
        
        node_impurity = self._calculate_impurity(y)
        
        for col in range(X.shape[1]):
            feature_level = np.unique(X[:, col])
            # TODO: не очень понимаю зач здесь так сделано, надо разобраться.
            thresholds = (feature_level[1:] + feature_level[:-1]) / 2.0
            
            for threshold in thresholds:
                y_left = y[X[:, col] <= threshold]
                left_impurity = self._calculate_impurity(y_left)
                N_left = y_left.shape[0] / y.shape[0]
                
                y_right = y[X[:, col] > threshold]
                right_impurity = self._calculate_impurity(y_right)
                N_right = y_right.shape[0] / y.shape[0]
                
                impurity_criterion = node_impurity - N_left * left_impurity \
                                    - N_right * right_impurity
                    
                if impurity_criterion > best_criterion:
                    best_criterion = impurity_criterion
                    best_feature = col
                    best_threshold = threshold
        
        if best_criterion == float('-inf'):
            return best_feature, best_threshold, False
                    
        return best_feature, best_threshold, True
        
    def _fit_node(self, X, y, node_id, depth):
        # Если выполнен критерий остановки, создать листовую вершину.
        # Был знк меньше.
        if (X.shape[0] <= self.min_samples_split or depth == self.max_depth):
            self.tree[node_id] = [self.Leaf, np.mean(y)]
            return
        
        feature_id, threshold, isOkay = self._search_split(X, y)
        
        # Значит скорее всего все фичи уже одинаковые в этом поддереве, двигаться некуда.
        if isOkay is False:
            self.tree[node_id] = [self.Leaf, np.mean(y)]
            return
        
        X_left, y_left = X[X[:, feature_id] >= threshold], y[X[:, feature_id] >= threshold]
        X_right, y_right = X[X[:, feature_id] < threshold], y[X[:, feature_id] < threshold]
        
        if (X_left.shape[0] < self.min_samples_leaf or
            X_right.shape[0] < self.min_samples_leaf):

            self.tree[node_id] = [self.Leaf, np.mean(y)]            
        else:
            self.tree[node_id] = [self.notLeaf, feature_id, threshold]
            self._fit_node(X_left, y_left, 2*node_id+1, depth+1)
            self._fit_node(X_right, y_right, 2*node_id+2, depth+1)
        
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        
        self._fit_node(X, y, 0, 0)
        
    def _predict(self, X, node_id):
        node_info = self.tree[node_id]
        answer = np.zeros(X.shape[0])

        if node_info[0] is self.notLeaf:
            feature_id, threshold = node_info[1], node_info[2]

            ids_left = np.where(X[:, feature_id] >= threshold)
            answer[ids_left] = self._predict(X[ids_left], 2*node_id+1)

            ids_right = np.where(X[:, feature_id] < threshold)
            answer[ids_right] = self._predict(X[ids_right], 2*node_id+2)
        else:
            answer = np.array([node_info[1]] * X.shape[0])
            
        return answer
            
        
    def predict(self, X):
        X = np.array(X)
        return self._predict(X, 0)

In [53]:
names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'name']
data = pd.read_csv('auto-mpg.data', delim_whitespace=True, names=names)

data['horsepower'].replace('?', -999, inplace=True)
data[['cylinders', 'year', 'origin', 'horsepower']] = data[['cylinders', 'year', 'origin', 'horsepower']
                                                          ].astype(np.float32)
data.drop(['origin', 'name'], axis=1, inplace=True)

X, y = data.iloc[:, 1:], data.iloc[:, 0]
X, y = np.array(X), np.array(y)

data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year
0,18.0,8.0,307.0,130.0,3504.0,12.0,70.0
1,15.0,8.0,350.0,165.0,3693.0,11.5,70.0
2,18.0,8.0,318.0,150.0,3436.0,11.0,70.0
3,16.0,8.0,304.0,150.0,3433.0,12.0,70.0
4,17.0,8.0,302.0,140.0,3449.0,10.5,70.0


In [54]:
kf = KFold(n_splits=8)

In [55]:
%%time
results = []
for train_id, test_id in kf.split(X, y):
    X_train, y_train = X[train_id], y[train_id]
    X_test, y_test = X[test_id], y[test_id]

    mytree_ex = MyDecisionTreeRegr()
    mytree_ex.fit(X_train, y_train)

    y_res = mytree_ex.predict(X_test)
    results.append(mean_squared_error(y_test, y_res))

print(results)
print(np.mean(results))

[4.135, 6.08345, 11.0075, 7.89755, 16.204099999999997, 33.9133, 39.26275510204081, 38.316071428571426]
19.602465816326532
CPU times: user 2.83 s, sys: 0 ns, total: 2.83 s
Wall time: 2.83 s


In [56]:
%%time
results = []
for train_id, test_id in kf.split(X, y):
    X_train, y_train = X[train_id], y[train_id]
    X_test, y_test = X[test_id], y[test_id]

    tree_ex = DecisionTreeRegressor()
    tree_ex.fit(X_train, y_train)

    y_res_ex = tree_ex.predict(X_test)
    results.append(mean_squared_error(y_test, y_res_ex))

print(results)
print(np.mean(results))

[4.36, 6.347799999999999, 10.635, 7.741999999999999, 18.238799999999998, 28.568, 44.39244897959185, 33.94938775510204]
19.279179591836733
CPU times: user 10.6 ms, sys: 0 ns, total: 10.6 ms
Wall time: 9.83 ms


Дерево вроде работает нормально, но долго, скорее всего из-за np.unique и все такое.

Дальше забабахаем градиентный бустинг на этих деревьях.

In [10]:
class constanta:
    def fit(self, X, y):
        self.result = np.mean(y)

    def predict(self, X):
        return self.result

In [11]:
class MyGradientBoostingRegr:
    def __init__(self, learning_rate=0.1, n_estimators=100, subsample=1.0,
                 min_samples_split=2, min_samples_leaf=1,
                 max_depth=3, alpha=0.9):
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators + 1  # +1 для константы.
        self.subsample = subsample
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.alpha = alpha
        
        self.estimators = []
        self.weights = []
        
    def _calculate_weight(self, G_ij, X, h_next):
        y_pred = h_next.predict(X)
        weights = G_ij / y_pred
        
        return np.mean(weights)
        
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        
        h_curr = constanta()
        h_curr.fit(X, y)
        w_curr = 1.0 * self.learning_rate
        self.estimators.append(h_curr)
        self.weights.append(w_curr)
        
        G_ij = y
        
        for i in range(1, self.n_estimators):
            y_pred = h_curr.predict(X) * w_curr
            G_ij = G_ij - y_pred
            
            h_next = MyDecisionTreeRegr(max_depth=3)
            h_next.fit(X, G_ij)
            
            w_next = self._calculate_weight(G_ij, X, h_next) * self.learning_rate
            
            self.estimators.append(h_next)
            self.weights.append(w_next)
            
            h_curr = h_next
            w_curr = w_next

    def predict(self, X):
        answer = np.zeros(X.shape[0])
        for i in range(self.n_estimators):
            answer += self.weights[i] * self.estimators[i].predict(X)
        return answer

In [ ]:
results_test = []

for i in range(5):
    kf = KFold(n_splits=5, shuffle=True, random_state=241)

    results = []
    for train_id, test_id in kf.split(X, y):
        X_train, y_train = X[train_id], y[train_id]
        X_test, y_test = X[test_id], y[test_id]

        my_gbr = MyGradientBoostingRegr()
        my_gbr.fit(X_train, y_train)

        y_res_ex = my_gbr.predict(X_test)
        results.append(mean_squared_error(y_test, y_res_ex))

    results_test.append(np.mean(results))

In [12]:
my_gbr = MyGradientBoostingRegr()
my_gbr.fit(X, y)

In [13]:
y_res = my_gbr.predict(X)
mean_squared_error(y, y_res)

2.288865138704365

In [14]:
gbr = GradientBoostingRegressor()
gbr.fit(X, y)

y_res = gbr.predict(X)
mean_squared_error(y, y_res)

2.2888648227232715

In [15]:
%%time
results = []
for train_id, test_id in kf.split(X, y):
    X_train, y_train = X[train_id], y[train_id]
    X_test, y_test = X[test_id], y[test_id]

    my_gbr = MyGradientBoostingRegr()
    my_gbr.fit(X_train, y_train)

    y_res_ex = my_gbr.predict(X_test)
    results.append(mean_squared_error(y_test, y_res_ex))

print(results)
print(np.mean(results))

[4.082612379988642, 2.8464555782722893, 6.487899218112115, 7.039800559129882, 9.22255892650585, 16.455416890274094, 20.879982613101955, 15.18765405801348]
10.275297527924788
CPU times: user 1min 5s, sys: 20.2 ms, total: 1min 5s
Wall time: 1min 5s


In [16]:
%%time
results = []
for train_id, test_id in kf.split(X, y):
    X_train, y_train = X[train_id], y[train_id]
    X_test, y_test = X[test_id], y[test_id]

    gbr = GradientBoostingRegressor()
    gbr.fit(X_train, y_train)

    y_res_ex = gbr.predict(X_test)
    results.append(mean_squared_error(y_test, y_res_ex))

print(results)
print(np.mean(results))

[3.823105169020543, 2.6904240712670786, 6.05231756973587, 7.2916631068352755, 10.031271119533464, 16.184696004243378, 29.486065887931968, 15.20998135881655]
11.346190535923014
CPU times: user 179 ms, sys: 0 ns, total: 179 ms
Wall time: 178 ms


Контрольная проверка, сравнение бустингов на 5 случайно пошафленных вариантов данных Auto-mpg, с валидацией по 5 фолдов.

In [18]:
results_test = []

for i in range(5):
    kf = KFold(n_splits=5, shuffle=True, random_state=241)

    results = []
    for train_id, test_id in kf.split(X, y):
        X_train, y_train = X[train_id], y[train_id]
        X_test, y_test = X[test_id], y[test_id]

        my_gbr = MyGradientBoostingRegr()
        my_gbr.fit(X_train, y_train)

        y_res_ex = my_gbr.predict(X_test)
        results.append(mean_squared_error(y_test, y_res_ex))

    results_test.append(np.mean(results))

In [19]:
results_valid = []

for i in range(5):
    kf = KFold(n_splits=5, shuffle=True, random_state=241)

    results = []
    for train_id, test_id in kf.split(X, y):
        X_train, y_train = X[train_id], y[train_id]
        X_test, y_test = X[test_id], y[test_id]

        gbr = GradientBoostingRegressor()
        gbr.fit(X_train, y_train)

        y_res_ex = gbr.predict(X_test)
        results.append(mean_squared_error(y_test, y_res_ex))

    results_valid.append(np.mean(results))

In [29]:
print("Результаты тестового градиентного бстинга:")
print(results_test)
print("\nРезультаты валидационного градиентного бстинга:")
print(results_valid)

Результаты тестового градиентного бстинга:
[7.710954090268361, 7.710954090268361, 7.710954090268361, 7.710954090268361, 7.710954090268361]

Результаты валидационного градиентного бстинга:
[7.710954090268361, 7.710954090268361, 7.710954090268361, 7.710954090268361, 7.710954090268361]


In [62]:
names = ['vendor', 'model', 'myct', 'mmin', 'mmax', 'cach', 'chmin', 'chmax', 'prp', 'erp']
data = pd.read_csv('machine.data', names=names)

data[['myct', 'mmin', 'mmax', 'cach', 'chmin', 'chmax', 'prp']] = data[['myct', 'mmin', 'mmax', 'cach',
                                                                        'chmin', 'chmax', 'prp']].astype(np.float32)
y_erp = data.iloc[:, -1]
data.drop(['vendor', 'model', 'erp'], axis=1, inplace=True)

X, y = data.iloc[:, :-1], data.iloc[:, -1]
X, y = np.array(X), np.array(y)

data.head()

,myct,mmin,mmax,cach,chmin,chmax,prp
0,125.0,256.0,6000.0,256.0,16.0,128.0,198.0
1,29.0,8000.0,32000.0,32.0,8.0,32.0,269.0
2,29.0,8000.0,32000.0,32.0,8.0,32.0,220.0
3,29.0,8000.0,32000.0,32.0,8.0,32.0,172.0
4,29.0,8000.0,16000.0,32.0,8.0,16.0,132.0


In [58]:
results_test = []

for i in range(5):
    kf = KFold(n_splits=5, shuffle=True, random_state=241)

    results = []
    for train_id, test_id in kf.split(X, y):
        X_train, y_train = X[train_id], y[train_id]
        X_test, y_test = X[test_id], y[test_id]

        my_gbr = MyGradientBoostingRegr()
        my_gbr.fit(X_train, y_train)

        y_res_ex = my_gbr.predict(X_test)
        results.append(mean_squared_error(y_test, y_res_ex))

    results_test.append(np.mean(results))

In [59]:
results_valid = []

for i in range(5):
    kf = KFold(n_splits=5, shuffle=True, random_state=241)

    results = []
    for train_id, test_id in kf.split(X, y):
        X_train, y_train = X[train_id], y[train_id]
        X_test, y_test = X[test_id], y[test_id]

        gbr = GradientBoostingRegressor()
        gbr.fit(X_train, y_train)

        y_res_ex = gbr.predict(X_test)
        results.append(mean_squared_error(y_test, y_res_ex))

    results_valid.append(np.mean(results))

In [60]:
print("Результаты тестового градиентного бстинга:")
print(results_test)
print("\nРезультаты валидационного градиентного бстинга:")
print(results_valid)

Результаты тестового градиентного бстинга:
[2888.2058171334575, 2888.2058171334575, 2888.2058171334575, 2888.2058171334575, 2888.2058171334575]

Результаты валидационного градиентного бстинга:
[3590.0607738249396, 3740.983603041978, 3955.9502994895533, 3790.802656980109, 3841.5475217413723]


In [63]:
# Для сравнения ошабка для предсказания от людей, выложивших датасет.
mean_squared_error(y, np.array(y_erp))

1737.3349282296651